In [ ]:
# HTML to Markdown
!pip install -r requirements.txt

In [ ]:
# 查看环境变量参数
%env ENV_FOR_DYNACONF=local

import os 
os.chdir('/Users/wangjialong/Documents/code/saic_project/global-vehicle-dragon/data-processing')

# 重新加载模块
from importlib import reload
from config.settings import Settings
import db.db_manager
import config.settings
import os

reload(config.settings)
reload(db.db_manager)
print(f"Debug: {bool(Settings.DEBUG)}")

file_directory = r'/Users/wangjialong/Documents/AI/SAIC_DCSGlobalVehicle/全球整车系统设计/GLBDCSVE'
output_directory = r'/Users/wangjialong/Documents/AI/SAIC_DCSGlobalVehicle/全球整车系统设计/GLBDCSVE_OUTPUT'
files = os.listdir(file_directory)
file_count = sum(os.path.isfile(os.path.join(file_directory, item)) for item in files)
print(f"目录 '{file_directory}' 下有 {file_count} 个文件。")

In [ ]:
# 将导出的confluence文档，进行处理，放到数据库中
from db.model.dragon_knowledge_base import DragonKnowledgeBase
import hashlib
from etl.dragon_etl import convert_to_md, save_model
from db.db_manager import DBConn

# 执行转换过程，并保存到数据库中
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for index, item in enumerate(files, start=1):
    black_list = ['.DS_Store']
    if item in black_list:
        continue;
    
    db_conn = DBConn()
    full_path = os.path.join(file_directory, item)
    new_item_name = item.replace('.html', '.md')
    hash_obj = hashlib.sha256()
    if os.path.isfile(full_path):
        print(f"处理文件 {index} |'{item}'")
        try:
            with open(full_path, 'r') as file:
                content = file.read()
                new_file_name = os.path.join(output_directory, new_item_name)
                md_result = convert_to_md(content);
                hash_obj.update(md_result.encode())
                new_model = DragonKnowledgeBase(name=new_item_name, \
                                            hash=hash_obj.hexdigest(), \
                                            content=md_result)
                # 数据库保存
                save_model(conn=db_conn, model=new_model)
                                            
                # 输出文件
                with open(new_file_name, 'w') as wfile:
                    wfile.write(md_result)                    
        except Exception as e:
            print(f"Failed! {index} | {full_path} | {e}")
    db_conn.close()
        